In [1]:
import math
import networkx as nx
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import torch_geometric.transforms as T
import geopandas as gpd
import pandas as pd
import momepy
import scipy as sp
import matplotlib.pyplot as plt

from utils.utils import remove_item
from utils.load_geodata import load_graph
from prediction import run, load_data
from utils.constants import *
!nvidia-smi

/vol/bitbucket/wwc4618/venv/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


Fri May 27 11:35:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 30%   31C    P8     7W / 180W |     80MiB /  8119MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
place = 'No Bounds'
model_dict = {}
result_dict = {}


for var_args in [
    # { 'num_neighbors': 2, 'batch_size': 1024 },
    # { 'split_approach': 'neighbor', 'num_neighbors': -1, 'batch_size': 2048 },
    { 'split_approach': 'saint', 'batch_size': 1024 },
    # { 'num_neighbors': 4, 'batch_size': 32 },
    # { 'num_neighbors': 8, 'batch_size': 32 },
    # { 'num_neighbors': 8, 'batch_size': 64 },
    # { 'num_neighbors': 8, 'batch_size': 128 },
    # {'model_type': 'gain', 'hidden_channels': 20, 'num_layers': 2,  'batch_norm': True, 'mlp_num_layers': 2},
    # {'model_type': 'gain', 'hidden_channels': 20, 'num_layers': 2,  'batch_norm': True, 'mlp_num_layers': 3},
    # {'model_type': 'gain', 'hidden_channels': 20, 'num_layers': 2,  'batch_norm': True, 'mlp_num_layers': 4},
    # {'model_type': 'gain', 'hidden_channels': 20, 'num_layers': 2,  'batch_norm': False, 'mlp_num_layers': 3},
]:  
    data_process_args = {
        
        'clean': False,
        'add_deg_feats': True,
        'include_feats': all_feature_fields,
        **var_args
    }
    model_args = {
        'model_type': 'gain',
        'hidden_channels': 10,
        'num_layers': 2,
    }
    print(f'Testing {var_args}')
    models, results = run(place, target_field='meridian_class',
                          data_process_args=data_process_args,
                          model_args=model_args,
                          lr=0.005,
                          num_iter=1)
    model_dict[str(var_args)] = models
    result_dict[str(var_args)] = results

Testing {'split_approach': 'saint', 'batch_size': 1024}
Loading graph of No Bounds with key ('No Bounds', "['metres', 'choice2km', 'nodecount2km', 'integration2km', 'choice10km', 'nodecount10km', 'integration10km', 'choice100km', 'nodecount100km', 'integration100km', 'choice2kmrank', 'choice10kmrank', 'integration10kmrank', 'integration2kmrank', 'choice2kmlog', 'choice10kmlog', 'choice100kmlog']", 'meridian_class', 'dual', False, True, 50, False)...
Loaded graph from saved files.
Classifying meridian_class
Encoded target into the following classes: ['aroad' 'broad' 'minor' 'motorway']
CE Loss with weights tensor([  4.7499,   7.5391,   0.6070, 106.3526])
Val Metrics when predicting all == 2:
{'accuracy_score': 0.8240036857866851, 'balanced_accuracy_score': 0.25, 'f1_score': 0.7444963839502331, 'matthews_corrcoef': 0.0}
Val Metrics when predicting all == 2:
{'accuracy_score': 0.8240036857866851, 'balanced_accuracy_score': 0.25, 'f1_score': 0.7444963839502331, 'matthews_corrcoef': 0.0}
St

,Train Loss,Accuracy,Macro Recall,F1,MCC
0,1.250707,0.480427,0.509963,0.566394,0.162467


In [ ]:
place = 'No Bounds'
result_dict = {}
for lr in [0.01,0.005,0.001]:
    data_process_args = {
        'split_approach': 'cluster',
        'include_feats': all_feature_fields,
    }
    model_args = {
        'model_type': 'gin',
        'hidden_channels': 10,
        'num_layers': 2,
    }
    print(f'Testing lr {lr}')
    models, results = run(place, target_field='accident_count',
                          data_process_args=data_process_args,
                          model_args=model_args,
                          lr=lr,
                          num_iter=3)
    result_dict[lr] = results

In [14]:
from matplotlib.lines import Line2D
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
import pylab


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
color_dict = {'motorway': '#d62728',
              'aroad': '#1f77b4',
              'broad': '#ff7f0e',
              'minor': '#7f7f7f'}

custom_lines = [Line2D([0], [0], color=col, lw=lw) for col, lw in zip(list(color_dict.values()), [5, 5, 5, 2])]

def visualize_preds(place, models, model_titles=None, only_pred=False, target_field = 'meridian_class'):
    data, _, _ = load_data(place, target_field,
                       include_feats=all_feature_fields,
                         add_deg_feats=True,
                        clean=False,
                        verbose=True)

    streets = load_graph(place, target_field=target_field,
                        feature_fields=all_feature_fields,
                         force_connected=True,
                         clean= False,
                        return_nx=True)

    # Convert nx graph to dgl (preserves node order) and get model predictions 
    data = data.to(device)
    bal, f1 = [], []
    for idx, model in enumerate(models):
        preds = model(data.x.float(), data.edge_index)
    
        # Convert preds to label and assign to nx nodes
        pred_dict = {}
        pred_classes = []
        for i, e in enumerate(streets.edges()):
            pred = preds[i].argmax().item()
            pred_classes.append(pred)
            pred_dict[e] = data.classes[int(pred)]
        nx.set_edge_attributes(streets, pred_dict, f'preds_model_{idx + 1}')

        # Obtain metrics
        bal.append(balanced_accuracy_score(data.y.cpu(), np.array(pred_classes)))
        f1.append(f1_score(data.y.cpu(), np.array(pred_classes), average='weighted'))
    
    # Convert nx graph back to gdf
    lines = momepy.nx_to_gdf(streets)[1]
    
    # Plotting
    title = f'{place}: {streets.number_of_nodes()} road segments'
    if only_pred:
        lines.plot(column='preds', cmap='tab10', figsize=(10, 10))
        plt.title(title)
        plt.show()
    else:
         # plot actual and predicted values
        num_plots = len(models) + 1
        f, ax = plt.subplots(1, num_plots, figsize=(num_plots*10, 10), sharex=True, sharey=True)
        target_fields = [target_field] + [f'preds_model_{idx + 1}' for idx in range(len(models))]
        for idx, target_field in enumerate(target_fields):
            minor_lines = lines[lines[target_field] == 'minor']
            other_lines = lines[lines[target_field] != 'minor']
            for g, lw, z in [(minor_lines, 2, 1), (other_lines, 5, 2)]:
                colors = [color_dict[cat] for cat in g[target_field]]
                g.plot(ax=ax[idx], column=target_field, colors=colors, linewidth=lw, zorder=z)
            
        for i, facet in enumerate(ax):
            if i == 0:
                subtitle = 'Actual Classes'
            else:
                idx = i - 1
                subtitle = f'Prediction by: {model_titles[idx]}\n' \
                     f'Macro-Average Recall: {bal[idx]:.3f}, Weighted F1: {f1[idx]:.3f}'
            facet.set_title(subtitle, size=20)
            facet.axis("off")
        f.legend(custom_lines, list(color_dict.keys()), loc='upper right', fontsize='xx-large', bbox_to_anchor=(0.9, 1), ncol=4)
        plt.suptitle(title, fontweight="bold", fontsize=30, x=0.3, y=0.98)
        plt.tight_layout()
        plt.show()

places = ['Westminster', 'Coventry', 'Birmingham']
model_titles = ['Neighbour', 'Cluster-GCN']
print(model_dict)
for place in places:
    models = [model_dict[model_type][0] for model_type in model_dict]
    visualize_preds(place, models, model_titles=model_titles, target_field='meridian_class')

{"{'split_approach': 'neighbor', 'num_neighbors': -1, 'batch_size': 2048}": [GAIN(22, 4, num_layers=2)], "{'split_approach': 'cluster'}": [GAIN(22, 4, num_layers=2)]}
Loading graph of Westminster with key ('Westminster', "['metres', 'choice2km', 'nodecount2km', 'integration2km', 'choice10km', 'nodecount10km', 'integration10km', 'choice100km', 'nodecount100km', 'integration100km', 'choice2kmrank', 'choice10kmrank', 'integration10kmrank', 'integration2kmrank', 'choice2kmlog', 'choice10kmlog', 'choice100kmlog']", 'meridian_class', 'dual', False, True, 50, False)...
Loaded graph from cache.
Classifying meridian_class
Encoded target into the following classes: ['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor'

Computing METIS partitioning...
Done!


['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'broad' 'minor']
['aroad' 'br

IndexError: index 3 is out of bounds for axis 0 with size 3

In [1]:

!jupyter nbconvert --to script predicting-classes.ipynb

[NbConvertApp] Converting notebook predicting-classes.ipynb to script
[NbConvertApp] Writing 13230 bytes to predicting-classes.py
